In [1]:
import os.path
from google.auth.transport.requests import Request
from google.oauth2.credentials import Credentials
from google_auth_oauthlib.flow import InstalledAppFlow
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError
import base64
import pandas as pd 
import time
from email.mime.text import MIMEText
import email
import traceback


In [2]:
SCOPES = ['https://www.googleapis.com/auth/gmail.readonly', 
          'https://www.googleapis.com/auth/gmail.modify']

creds = Credentials.from_authorized_user_file('token.json', SCOPES)
service = build('gmail', 'v1', credentials=creds)

In [3]:
#Função que escreve os emails em um arquivo txt
def get_emails():
    with open('emails.txt', 'w', encoding='utf-8') as f:
        try:
            service = build('gmail', 'v1', credentials=creds)
            results = service.users().messages().list(userId='me', maxResults=10).execute()
            messages = results.get('messages', [])

            for message in messages:
                msg = service.users().messages().get(userId='me', id=message['id']).execute()
                payload = msg['payload']
                headers = payload['headers']
                
                #Escreve o remetente, o assunto e a data
                for header in headers:
                    if header['name'] == 'From':
                        f.write(f"From: {header['value']}\n")
                    if header['name'] == 'Subject':
                        f.write(f"Subject: {header['value']}\n")
                    if header['name'] == 'Date':
                        f.write(f"Date: {header['value']}\n")
                
                #Se o corpo do email está vazio ou não segue o padrao MIME, a mensagem abaixo é escrita e o loop volta
                if 'parts' not in payload:
                    f.write("e-mail não contem nenhuma parte com o tipo MIME\n")
                    f.write("---------------------------------------------------------\n")    
                    continue
            
                #Escreve o corpo do email
                parts = payload['parts']
                for part in parts:
                    if part['mimeType'] == 'text/plain':
                        body = part['body']['data']
                        decoded_body = base64.urlsafe_b64decode(body).decode('utf-8')
                        f.write(f"Body: {decoded_body}\n")

                f.write("---------------------------------------------------------\n")
        except HttpError as error:
            f.write(f"An error occurred: {error}\n")
            messages = []

    return messages

get_emails()

[{'id': '18768307d8edfbb3', 'threadId': '18768307d8edfbb3'},
 {'id': '187683018e887241', 'threadId': '187683018e887241'},
 {'id': '187682fb5fba71c7', 'threadId': '187682fb5fba71c7'},
 {'id': '187682f51ca0fd38', 'threadId': '187682f51ca0fd38'},
 {'id': '187682eed211bb6b', 'threadId': '187682eed211bb6b'},
 {'id': '187682e88037ce28', 'threadId': '187682e88037ce28'},
 {'id': '187682e24441f3ed', 'threadId': '187682e24441f3ed'},
 {'id': '187682dc0fd6d853', 'threadId': '187682dc0fd6d853'},
 {'id': '187682d5cb6e1cbe', 'threadId': '187682d5cb6e1cbe'},
 {'id': '187682cf862166ce', 'threadId': '187682cf862166ce'}]

In [5]:
def searchStringMessage(service, userID, searchString):
    """
    Esta função irá retornar a lista com ID das mensagem que possuem a string (palavra) em questão
    Args:
        service: autenticação da API
        userID: id do usuário, no caso, o cliente, será sempre "me"(eu)
        searchString: apenas retornará as mensagem com o texto ou palavra presente presente
    Returns: lista com todos os IDs com a searchString em específico.
    """
    try: 
        idList = [] #inicializando a lista
        
        #adquirindo a mensagem (aonde a mágica acontece!)
        searchIds = service.users().messages().list(userId=userID, q=searchString).execute()
        
        try:
            EmailIds = searchIds ['messages']
        except KeyError:
            print("retornado 0 resultados")
            return idList
        
        if len(EmailIds)>=1:    #anexa todos os emails na lista "idList"
            for msg_id in EmailIds:
                idList.append(msg_id['id'])
            return (idList)
    except HttpError as error:
        print("ocorreu um erro: %s", error) 

In [6]:
def viewMessage(service, userID, msgID):
    """
    Esta função mostra os emails presente na lista de ID
    Args:
        service: autenticação da API
        userID: id do usuário, no caso, o cliente, será sempre "me"(eu)
        msgID: iD da mensagem em questão
    Returns: o email em sua parte texto e html
    """
    try:
        message = service.users().messages().get(userId=userID, id=msgID, format='raw').execute() #retorna a mensagem em formato raw codificado em base64
        mime_msg = email.message_from_bytes(base64.urlsafe_b64decode(message['raw']))
        
        print("-----")        
        contentType = mime_msg.get_content_maintype()
        if contentType == 'multipart':
            for part in mime_msg.get_payload():
                if part.get_content_maintype() == 'text':
                    print(part.get_payload())
        elif contentType == 'text':
            print(mime_msg.get_payload())
        print("-----")
        
    except Exception as error:
        print("ocorreu um erro: %s", error)

In [7]:
def viewMessageSnippet(service, userID, msgID):
    """
    Esta função mostra os emails presente na lista de ID, apenas o subject e o snippet da mensagem
    Args:
        service: autenticação da API
        userID: id do usuário, no caso, o cliente, será sempre "me"(eu)
        msgID: iD da mensagem em questão
    Returns: o snippet do email
    """
    try:
        message = service.users().messages().get(userId=userID, id=msgID, format='raw').execute() #retorna a mensagem em formato raw codificado em base64
        mime_msg = email.message_from_bytes(base64.urlsafe_b64decode(message['raw']))
        
        #print(mime_msg['from']) mostrar o email do remetente       
        #print(mime_msg['to'])  mostrar o email do destinatário
        print(mime_msg['subject'])
        print(message['snippet'])
        print(' ')
        
    except Exception as error:
        print("ocorreu um erro: %s", error) 

In [8]:
userID = 'me'
words = ['REEMBOLSO', 'CONTA', 'ENTREGA', 'CONTATO', 'PAGAMENTO', 'FATURA', 'CANCELAMENTO', 'RASTREAR', 'NOTÍCIAS', 'OPINIÃO']
IDs = {}


for word in words:
    IDs_palavras = searchStringMessage(service, userID, word)
    IDs[word] = IDs_palavras

#visualizar as mensagens de cada ID.
for word, ids in IDs.items():
    print(f'A palavra \'{word}\' tem {len(ids)} IDs correspondentes.')


for word, ids in IDs.items():
    print(word, ids)

A palavra 'REEMBOLSO' tem 16 IDs correspondentes.
A palavra 'CONTA' tem 49 IDs correspondentes.
A palavra 'ENTREGA' tem 33 IDs correspondentes.
A palavra 'CONTATO' tem 10 IDs correspondentes.
A palavra 'PAGAMENTO' tem 15 IDs correspondentes.
A palavra 'FATURA' tem 9 IDs correspondentes.
A palavra 'CANCELAMENTO' tem 9 IDs correspondentes.
A palavra 'RASTREAR' tem 6 IDs correspondentes.
A palavra 'NOTÍCIAS' tem 8 IDs correspondentes.
A palavra 'OPINIÃO' tem 11 IDs correspondentes.
REEMBOLSO ['18768307d8edfbb3', '187682f51ca0fd38', '1876829727fdf59d', '1876825267899f5d', '18768213f3eeb60e', '187681e1ed969f0d', '18768196e3eeeb21', '187680fa773b0464', '187680a8ff7fde48', '1876807d3ea1e4b8', '1876802bab007ac1', '1876801f2c0f33e8', '18767fc7aab8a5e3', '18767fbb3d4f35fe', '1872a61a746b36ea', '1872a5edd6237caf']
CONTA ['187682c92439722a', '187682a3b4702937', '18768290e8e28730', '187682847b1e414c', '1876827e0dd30e5f', '18768271b4356f00', '18768258b296d44a', '1876824c286a1ef6', '1876823f91426bda'

In [9]:

created_label = {}

for word in words:
    label_name = word
    label = {'name': label_name, 'labelListVisibility': 'labelShow'}
    created_label[label_name] = service.users().labels().create(userId='me', body=label).execute()
    print(f'Rótulo "{label_name}" criado com sucesso!')



HttpError: <HttpError 409 when requesting https://gmail.googleapis.com/gmail/v1/users/me/labels?alt=json returned "Label name exists or conflicts". Details: "[{'message': 'Label name exists or conflicts', 'domain': 'global', 'reason': 'aborted'}]">

In [114]:
print(created_label)
label_id = []
for word in words:
    label_id.append(created_label[word]['id'])

print(label_id)


{'REEMBOLSO': {'id': 'Label_61', 'name': 'REEMBOLSO', 'messageListVisibility': 'show', 'labelListVisibility': 'labelShow'}, 'CONTA': {'id': 'Label_62', 'name': 'CONTA', 'messageListVisibility': 'show', 'labelListVisibility': 'labelShow'}, 'ENTREGA': {'id': 'Label_63', 'name': 'ENTREGA', 'messageListVisibility': 'show', 'labelListVisibility': 'labelShow'}, 'CONTATO': {'id': 'Label_64', 'name': 'CONTATO', 'messageListVisibility': 'show', 'labelListVisibility': 'labelShow'}, 'PAGAMENTO': {'id': 'Label_65', 'name': 'PAGAMENTO', 'messageListVisibility': 'show', 'labelListVisibility': 'labelShow'}, 'FATURA': {'id': 'Label_66', 'name': 'FATURA', 'messageListVisibility': 'show', 'labelListVisibility': 'labelShow'}, 'CANCELAMENTO': {'id': 'Label_67', 'name': 'CANCELAMENTO', 'messageListVisibility': 'show', 'labelListVisibility': 'labelShow'}, 'RASTREAR': {'id': 'Label_68', 'name': 'RASTREAR', 'messageListVisibility': 'show', 'labelListVisibility': 'labelShow'}, 'NOTÍCIAS': {'id': 'Label_69', 'n

In [116]:
i = 0

for word, ids in IDs.items():
    labelId = label_id[i]
    i += 1
    for message_id in ids:
        label_body = {
            'removeLabelIds': ['IMPORTANT', 'CATEGORY_UPDATES', 'INBOX'],
            'addLabelIds': [labelId]
        }
        service.users().messages().modify(userId='me', id=message_id, body=label_body).execute()


In [86]:
labels = service.users().labels().list(userId='me').execute()
for label in labels['labels']:
    print(label['name'])

CHAT
SENT
INBOX
IMPORTANT
TRASH
DRAFT
SPAM
CATEGORY_FORUMS
CATEGORY_UPDATES
CATEGORY_PERSONAL
CATEGORY_PROMOTIONS
CATEGORY_SOCIAL
STARRED
UNREAD
MinhaCategoria
REEMBOLSO
CONTA
ENTREGA
CONTATO
PAGAMENTO
FATURA
CANCELAMENTO
RASTREAR
NOTÍCIAS
OPINIÃO
